# **Effect of CoinMarketCap Listing on Cryptocurrency ROIs**

Cryptocurrencies have made a name for themselves as the modern age gold rush, and everyday more investor success stories can be read about to keep us dreaming.
However it is extremely difficult to know when to invest and if it is worth investing in cryptocurrencies at all as we hear a lot about the successful investors but seldomly about those who lost their entire capital.

In this study,we will attempt to figure out if there is a correlation between return on investment (ROI thereafter) and time elapsed after a cryptoasset is listed on [CoinMarketCap](https://coinmarketcap.com/).

In order to tackle this question, we will make use of all the data available on [CoinMarketCap](https://coinmarketcap.com/), the world's most-referenced price-tracking website for cryptoassets.
This data was scrapped by [Bisola Olasehinde](https://www.kaggle.com/bizzyvinci) and is available [here](https://www.kaggle.com/bizzyvinci/coinmarketcap-historical-data) and is comprised of about 4.4 million datapoints.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
from os import listdir

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from datetime import datetime
from datetime import timedelta

The first step in our analysis will be to load the `coin.csv` file and filter our duplicate coins as well as coins that could negatively impact our analysis (coins with extremely low daily volume and very sharp price changes).

In [2]:
#load the list of coins on CoinMarketCap
coin_list = pd.read_csv('../input/coinmarketcap-historical-data/coins.csv')
cols = ['id','name', 'symbol', 'status','date_added']

coin_list = coin_list[cols]

#get rid of duplicate entries
coin_list = coin_list.drop_duplicates(subset=['id'], keep='first')
coin_list = coin_list.set_index('id')


#remove utracked and extinct for lack of data
coin_list = coin_list[coin_list['status'] != 'untracked']
coin_list = coin_list[coin_list['status'] != 'extinct']

#also drop a few problematic coins from the dataset (very low volume pump-and-dump schemes)
drop_list = ["BTC3L","BTC3S","BTCSHORT","ETH3L","ETH3S","INNBC",
            "INNBCL", "PHX", "LOOP", "LCT", "WOW", "ELONGD",
            "WSPP", "PAC", "ZUR", "PRIV", "GCC","ELCO",
            "RPD", "N8V"]

coin_list = coin_list[~coin_list['symbol'].isin(drop_list)]

#sort them by date added
coin_list=coin_list.sort_values(by='date_added', ascending=True)

#get filtered coin list to csv
coin_list.to_csv("filtered_coins.csv",index=True)

#create series coin_id->coin_name
coin_name_list = coin_list["name"]

#create series coin_name->coin_id
coin_id_list = pd.Series(coin_name_list.index.values, index=coin_name_list)


Now that we have a clean list of coins to work with, it's time to load and clean `historical.csv` which contains all the closing prices of all coins on [CoinMarketCap](https://coinmarketcap.com/) from 04/28/2013 to 07/31/2021.
For this analysis, since we are trying to figure out if the age of a coin correlates with its return on investment, we will (mostly) disregard the date of creation of the cryptoassets and instead focus on the amount of days elapsed since the assets were listed on CoinMarketCap.
This will make comparing these currencies to one another easier.

In [3]:
#load the entirety of CoinMarketCap's historical data
cmc_historical_data = pd.read_csv('../input/coinmarketcap-historical-data/historical.csv')

#only keep the date, the coin id and the coin price for each datapoint
cols = ['date','coin_id', 'price']

cmc_historical_data = cmc_historical_data[cols]

#all analysis relies on price column, so delete empty price rows or no id
cmc_historical_data = cmc_historical_data.dropna(subset=['coin_id', 'price'])
last_date = cmc_historical_data['date'].iloc[-1]

last_date = pd.to_datetime(last_date)

#for each coin, we want to have ROI from listing day to day of data scrapping (2021-07-31)
#there's a maximum of 3017 different dates between btc listing date and scrapping of this data

#create a list containing 3017 elements to use as the index of our dataframe
days = []
for i in range(3017):
    days.append(i)

#start a dataframe containing all the ROIs sorted by date
all_coins_ROI = pd.DataFrame(days, columns=['elapsed time'])

#loop over each coin, get first date and first coin price,
#then substract first date to each datetime to get time difference
#and divide each price by initial price to get ROI

all_coin_ids = cmc_historical_data['coin_id'].unique()

all_coin_starting_date=[]
rejected_coins = []
coin_names = []

for coin_id in coin_name_list.index:
        #only keep data that match this coin id
        coin_data = cmc_historical_data[cmc_historical_data['coin_id'] == coin_id]
        #sort remaining data from oldest to most recent
        coin_data = coin_data.sort_values(by='date', ascending=True)
        #convert date text to datetime object
        coin_data['date'] = pd.to_datetime(coin_data['date'])        
        
        #get coin name from the coin_name_list list
        coin_name = coin_name_list[coin_id]
        
        #get date of last datapoint
        coin_last_datapoint = coin_data['date'].iloc[-1]
        
        #declare empty lists
        extra_datapoints = []
        extra_datapoint = []
        
        #if coin_last_datapoint is different from 07-31-2021,
        #then it is assumed the coin died prematurely
        #pack the dataframe with 0s until the last datapoint is 07-31-2021
        if coin_last_datapoint != last_date:
            current_datepoint = coin_last_datapoint
            while current_datepoint < last_date:
                current_datepoint = current_datepoint + timedelta(days=1)
                extra_datapoint = [current_datepoint, coin_id, 0]
                extra_datapoints.append(extra_datapoint)
            
            extra_datapoints_df = pd.DataFrame(extra_datapoints, columns=coin_data.columns)
            
            coin_data = pd.concat([coin_data,extra_datapoints_df])

        #find the listing price of the coin (cannot be 0)
        coin_starting_price = 0
        try:
            for i in range(10):
                if coin_starting_price>0:
                    coin_starting_date = coin_data['date'].iloc[i-1]
                    #print(f'start price = {coin_starting_price} and {coin_starting_date}')
                    break
                else:
                    coin_starting_price = coin_data['price'].iloc[i]
        #if after 10 tries the price is still 0,
        #then it is assumed something is wrong with that coin
        #so skip it and go to next coin
        except:
            rejected_coins.append(coin_id)
            continue

        #if the first price of the coin is 0 or nan, then it is assumed something is wrong with that coin
        #so skip it and go to next coin
        if coin_starting_price == 0 or np.isnan(coin_starting_price):
            #keep track of all errored coins to remove them from df later
            rejected_coins.append(coin_id)
            continue
        else:
            #add the first date of the coin to the list
            all_coin_starting_date.append(coin_starting_date)

            #change datapoints' dates to the datapoints' dates - first date
            coin_data['date']=(coin_data['date']-coin_starting_date).dt.days
            #likewise, convert price to ROI
            coin_data['price']=coin_data['price']/coin_starting_price
            
            #in case of duplicate dates, keep the first entry and delete the others
            coin_data = coin_data.drop_duplicates(subset='date',keep='first')
            
            #check the median ROI for the first 10 points 
            first_coin_prices=coin_data['price'].head(10)
            median_first_prices = first_coin_prices.median()
            
            #if the median for the first 10 ROI is more than 100x the first price,
            #replace first price by the median price
            #this will insulate us against bad first entry points
            if median_first_prices>100*coin_starting_price:
                coin_starting_price = median_first_prices
                coin_data['price']=coin_data['price']/coin_starting_price
                
            #if one datapoint is bigger than 200x previous day or 1000x more than 2 days ago 
            #and previous day is >0, drop datapoint to protect us from low volume PnD
            coin_data['200_shift'] = 200*coin_data['price'].shift()
            coin_data['1000_shift'] = 200*coin_data['price'].shift(2)
            coin_data['200_diff'] = coin_data['200_shift'] - coin_data['price']
            coin_data['1000_diff'] = coin_data['1000_shift'] - coin_data['price']
            
            coin_data = coin_data[~((coin_data['200_diff']<0) & (coin_data['200_shift'] >0)|
                                                          (coin_data['1000_diff']<0) & (coin_data['1000_shift'] >0))]
            
            #we can now get rid of the two previously created columns since we are done dropping PnD points
            coin_data.drop(['200_shift', '200_diff','1000_shift','1000_diff'], axis=1)

            coin_data = coin_data.set_index('date')
            coin_data.index.name = None
            
            #add the filtered coin data to our df
            all_coins_ROI[coin_name] = coin_data['price']
            coin_names.append(coin_name)
    

#update the list of coins still in our analysis
coin_name_list = coin_name_list.drop(index=rejected_coins)
coin_id_list = coin_id_list[~coin_id_list.index.isin(rejected_coins)]

#find the listing year for each coin in our analysis
coin_startdate_df = pd.DataFrame(data=all_coin_starting_date,index = coin_name_list)
coin_startdate_df.columns =["year"]
coin_startdate_df["year"]=coin_startdate_df["year"].dt.year
    
all_coins_ROI=all_coins_ROI.set_index('elapsed time')  
#defragment
all_coins_ROI=all_coins_ROI.copy()

#calculate the mean and median of all coins for all dates    
all_coins_ROI['mean'] = all_coins_ROI.mean(axis=1,skipna=True)
all_coins_ROI['median'] = all_coins_ROI.median(axis=1,skipna=True)

#drop rows with no Bitcoin data.
#consindering btc status, if there's a missing datapoint 
#it's more likely an issue with the specific row index
#than an actual missing datapoint
all_coins_ROI = all_coins_ROI.dropna(subset=['Bitcoin'])

#Transposing the data will make it easier to draw bar charts for the coins
all_coins_ROI_T = all_coins_ROI.T

#all_coins_ROI.to_csv("all_coins_sorted.csv",index=True)
#all_coins_ROI_T.to_csv("all_coins_sorted_transposed.csv",index=True)

With the data now cleaned, we can start visualizing and processing it.
First let's visualize the mean and median normalized price of all coins (meaning the starting price of all coins is set to 1 and subsequent prices are proportional to that starting price) as a function of the time elapsed since they were listed on CoinMarketCap, as well as the year at which they were listed on the platform.

In [4]:
coin_startdate_df.index.names = ['coin']
coin_startdate_df = coin_startdate_df[~coin_startdate_df.index.duplicated(keep='first')]

#show a graph displaying how many coins were created each year
fig = px.histogram(coin_startdate_df, x="year")
#add an annotation in the top left corner showing total coin count
fig.add_annotation(x=2013, y=2300,text=f'Total Coin Count = {len(coin_startdate_df.index)}',
                   showarrow=False,yshift=10,align="left")
fig.update_layout(bargap=0.2)
fig.update_xaxes(title_text="Cryptocurrency Listing Year")
fig.update_yaxes(title_text="Cryptocurrency Count")
fig.update_layout(title_text="Count of Analyzed Cryptocurrencies Categorized"
                  "by their Listing Year on CoinMarketCap")
fig.show()

#create graph object showing all coins' mean and median ROIs in function of days elapsed since they were listed
mean_fig = px.line(all_coins_ROI,  y="mean")
median_fig = px.line(all_coins_ROI,  y="median")

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into its traces
mean_fig_traces = []
median_fig_traces = []
for trace in range(len(mean_fig["data"])):
    mean_fig_traces.append(mean_fig["data"][trace])
for trace in range(len(median_fig["data"])):
    median_fig_traces.append(median_fig["data"][trace])

#Create a 1x2 subplot
fig = sp.make_subplots(rows=1, cols=2,subplot_titles=("Mean Normalized Price", 
                                                      "Median Normalized Price")) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in mean_fig_traces:
    fig.append_trace(traces, row=1, col=1)
for traces in median_fig_traces:
    fig.append_trace(traces, row=1, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Elapsed Time (Days)", row=1, col=1)
fig.update_xaxes(title_text="Elapsed Time (Days)", row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Normalized Price", row=1, col=1)
fig.update_yaxes(title_text="Normalized Price", row=1, col=2)

fig.update_layout(title_text="Side By Side Mean and Median Normalized Prices"
                  "as a function of elapsed days since listing on CoinMarketCap")
fig.show()

From the first plot we can notice that more than half of the coins in this analysis were created after 01/01/2020, with a third of all coins created within the first 7 months of 2021, highlighting the recent rise in  popularity of cryptocurrencies.

From the two following plots we notice a drastic difference between average and median normalized prices, which could be explained by a few outlier cryptoassets with very high ROIs skewing the mean price up. 
More concerning even is the rate at which the median cryptoasset price converges to 0 (-100% ROI), highlighting the fact that the majority of cryptocurrencies rapidly lose money after being listed on CoinMarketCap.
The reason for the very sharp median normalized price increase around the 3000 days mark is due to the recency of the cryptocurrency invention: very few coins are that old, and as such the number of coins in the analysis abruptly plummet around the 8 year mark.

In order to confirm our hypothesis, we will sort the coins as a function of their ROI for four different hosen milestones: 30 days after listing, 120 days after listing, 1 year after listing and finally 3 years after listing.
Two thirds of the coins in this analysis have been listing for 3 years or less, so plotting the ROI for further milestones could yield less statistically significant results.

In [5]:
#create empty dataframe
#this dataframe will store binned ROIs at a specific moment after coin listing
selected_dates_ROI_df = pd.DataFrame({'A' : []})

cols = [29,30,31] #3 days around 30th day after listing, for average purpose
selected_dates_ROI_df['30 days ROI'] = all_coins_ROI_T[cols].mean(axis=1,skipna=True)
cols = [118,119,120,121,122] #5 days around 120th
selected_dates_ROI_df['120 days ROI'] = all_coins_ROI_T[cols].mean(axis=1,skipna=True)
cols = range(360,371) #10 days around 1st anniversary
selected_dates_ROI_df['1 year ROI'] = all_coins_ROI_T[cols].mean(axis=1,skipna=True)
cols = range(1085,1096) #10 days around 3rd anniversary
selected_dates_ROI_df['3 years ROI'] = all_coins_ROI_T[cols].mean(axis=1,skipna=True)

#delete empty column A
selected_dates_ROI_df=selected_dates_ROI_df.drop(columns=['A'])

#extract the mean and median from selected_dates_ROI_df
mean_median_ROI_df = selected_dates_ROI_df.iloc[-2:]

#change df's index and remove potential duplicates
selected_dates_ROI_df.index.names = ['coin']

#create bin labels and delimiters for binning the ROIs
bin_labels = ['-100%', '-100%_-90%', '-90%_-50%','-50%_0%','0%_100%',
              '100%_1,000%','1,000%_10,000%','10,000%_100,000%','100,000%+']
new_bins = [0,0.0001,0.1,0.5,1,2,11,101,1001,10001]

#duplicate df to have a binned and a non-binned version
binned_selected_dates_ROI_df = selected_dates_ROI_df.copy()

#bin the columns
binned_selected_dates_ROI_df['30 days ROI'] = pd.cut(
    binned_selected_dates_ROI_df['30 days ROI'],
    bins = new_bins,labels=bin_labels,precision=4,
    include_lowest=True
)

binned_selected_dates_ROI_df['120 days ROI'] = pd.cut(
    binned_selected_dates_ROI_df['120 days ROI'],
    bins = new_bins,labels=bin_labels,precision=4,
    include_lowest=True
)

binned_selected_dates_ROI_df['1 year ROI'] = pd.cut(
    binned_selected_dates_ROI_df['1 year ROI'],
    bins = new_bins,labels=bin_labels,precision=4,
    include_lowest=True
)

binned_selected_dates_ROI_df['3 years ROI'] = pd.cut(
    binned_selected_dates_ROI_df['3 years ROI'],
    bins = new_bins,labels=bin_labels,precision=4,
    include_lowest=True
)

#concat the non-binned and binned df and the coins' listing year
result = pd.concat([coin_startdate_df, selected_dates_ROI_df], axis=1)
binned_result = pd.concat([coin_startdate_df, binned_selected_dates_ROI_df], axis=1)

#convert year from str to int
result["year"] = pd.to_numeric(result["year"],downcast="integer")
result["30 days ROI"] = pd.to_numeric(result["30 days ROI"],downcast="float")
binned_result["year"] = pd.to_numeric(binned_result["year"],downcast="integer")


With the data processed, we can now plot the data for the four previously chosen milestones.
For each milestone we will plot a bar graph displaying the coins ROI binned into one of nine categories, as well as a line chart of all coins ROI.

In [6]:
#plot a binned df's column as a bar chart
def plot_binned_ROI(df,col_name):
    year_bin_list = []
    bin_count_list = []
    
    #initialize the plot
    fig = go.Figure()

    #for each year, count how many coins falls into each bin
    for year in range(2013,2022): #year_list:
        bin_list = []
        for bin in bin_labels:
            bin_count=len(df.index[(df[col_name] == bin) & (df["year"] == year)].tolist())
            bin_list.append(bin_count)
            
        year_bin_list.append(bin_list)
        bin_count_list.append(sum(bin_list))

        #create one bar per year showing the count in each bin
        fig.add_trace(go.Bar(x=bin_labels, y=year_bin_list[year-2013], name=year,
                        hovertemplate="Listing Year=%s<br>Coin Count=%%{y}<extra></extra>"
                             % year))
    
    #stack the year bars
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis_title=col_name,yaxis_title="Cryptoasset Count")
    fig.update_layout(legend_title_text = "Listing Year")
    
    plot_title = col_name[:-4]
    fig.update_layout(title=f'Cryptocurrencies sorted by ROI and listing year' 
                      f'{plot_title} after listing on CoinMarketCap')
    
    max_count = 0
    #figure out the count for each stacked bar to know where to place the % annotation
    for i in range(9):
        count = 0
        for year in range(9):
            count += year_bin_list[year][i]
        fig.add_annotation(x=i, y=count,
                text=f'{count} ({str(round(count/sum(bin_count_list)*100, 2))}%)',
                showarrow=False,
                yshift=10)
        if count > max_count:
            max_count = count
    
    #add text in the top right corner showing the median and average ROI for each graph
    fig.add_annotation(x=8, y=max_count*.9,
                text=f'Mean ROI = {str(round(mean_median_ROI_df[col_name][0]*100, 2))}%<br>'
                       f'Median ROI = {str(round(mean_median_ROI_df[col_name][1]*100, 2))}%',
                showarrow=False,
                yshift=10,
                align="right")
    
    fig.show()

def plot_individual_ROI(df,col_name):
    #sort the column in ascending order
    df=df.sort_values(by=col_name, ascending=True)
    #data_to_plot = df[col_name].dropna()
    data_to_plot = df.copy()
    #remove coins that don't have data to plot
    data_to_plot = data_to_plot.dropna(subset=[col_name])
    
    data_to_plot[col_name] = data_to_plot[col_name]-1
    
    #find where ROI = 0
    zero = np.where(data_to_plot[col_name] > -0.999)[0][0]
    
    #find where ROI over 1
    unity = np.where(data_to_plot[col_name] >0)[0][0]
    
    fig = px.line(data_to_plot, y=col_name, title='Individual Coin ROIs')
    fig.add_vline(x=zero, line_width=2, line_dash="dash", line_color="green", 
                  annotation_text="Dead Coins",
                  annotation_position="top left",
                  annotation_textangle=-90,
                  annotation_font_size=15,
                  annotation_font_color="green")
    fig.add_vline(x=zero, line_width=2, line_dash="dash", line_color="green", 
                  annotation_text="Negative ROI", 
                  annotation_position="top right",
                  annotation_textangle=-90,
                  annotation_font_size=15,
                  annotation_font_color="green")
    fig.add_vline(x=unity, line_width=2, line_dash="dash", line_color="green", 
                  annotation_text="Negative ROI", 
                  annotation_position="top left",
                  annotation_textangle=-90,
                  annotation_font_size=15,
                  annotation_font_color="green")
    fig.add_vline(x=unity, line_width=2, line_dash="dash", line_color="green", 
                  annotation_text="Positive ROI", 
                  annotation_position="top right",
                  annotation_textangle=-90,
                  annotation_font_size=15,
                  annotation_font_color="green")
    
    plot_title = col_name[:-4]
    fig.update_layout(title=f'Cryptoasset ROIs {plot_title} after listing on CoinMarketCap',
                   xaxis_title='Coins',
                   yaxis_title='Coin ROIs')
    fig.update_xaxes(showticklabels=False) # hide all the xticks
    
    fig.show()
    
#plot each graph
plot_binned_ROI(binned_result,'30 days ROI')
plot_individual_ROI(result,'30 days ROI')

plot_binned_ROI(binned_result,'120 days ROI')
plot_individual_ROI(result,'120 days ROI')

plot_binned_ROI(binned_result,'1 year ROI')
plot_individual_ROI(result,'1 year ROI')

plot_binned_ROI(binned_result,'3 years ROI')
plot_individual_ROI(result,'3 years ROI')

 

These plots confirm our hypothesis that the majority of cryptoassets have a negative return on investment shortly after being listed on CoinMarketCap, with more than half of cryptocurrencies being worthless within three years of their listing. We also notice that a very small number of coins have extremely high returns (some over 2,500,000% return within a year of listing) which greatly skew the average ROI one can expect.

It can be concluded that cryptocurrencies remain an extremely volatile investment with a high risk, high reward profile.